### Model versioning and registration with MLFlow


In [4]:
import os 
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging 

logging.basicConfig(level = logging.WARN)
logger = logging.getLogger(__name__)

> Note on ElasticNet: ElasticNet is a linear regression model that combines the properties of both Ridge regression and Lasso regression. It is a regularization technique used for regression tasks, particularly when dealing with high-dimensional datasets where the number of features (variables) is larger than the number of observations.

The ElasticNet model introduces two hyperparameters: alpha and l1_ratio.

* Alpha controls the overall strength of regularization. A higher alpha value results in stronger regularization, which shrinks the regression coefficients towards zero more aggressively.
* The l1_ratio determines the balance between the L1 and L2 penalties. A l1_ratio of 1 corresponds to pure Lasso regression, while a l1_ratio of 0 corresponds to pure Ridge regression. Intermediate values allow ElasticNet to mix the properties of both Lasso and Ridge.


In [5]:
def eval_metric(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)

    return rmse, mae, r2

In [6]:
if __name__ == "__main__":

    warnings.filterwarnings('ignore')
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )

    try:
        data = pd.read_csv(csv_url, sep = ';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )
    
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scaler from [3, 9]
    train_x = train.drop(['quality'], axis = 1)
    test_x = test.drop(['quality'], axis = 1)
    train_y = train[['quality']]
    test_y = test[['quality']]

    alpha = 0.5
    l1_ratio = 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha = alpha, l1_ratio = l1_ratio, random_state = 42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metric(test_y, predicted_qualities)

        print("ElasticNet model (alpha-%f, l1_ratio-%f):" % (alpha, l1_ratio))
        print("RMSE: %s" % rmse)
        print('MAE: %s' %mae)
        print("R2: %s" %r2)

        mlflow.log_param('alpha', alpha)
        mlflow.log_param('l1 _ratio', l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric('mae', mae)

        tracking_url_type_store = urlparse(mlflow.get_registry_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != 'file':

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # Please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name = "ElasticnetWineModel")

        else:
            mlflow.sklearn.log_model(lr, "model")

ElasticNet model (alpha-0.500000, l1_ratio-0.500000):
RMSE: 0.793164022927685
MAE: 0.6271946374319586
R2: 0.10862644997792636


In [11]:
!mlflow ui

[2023-06-28 16:29:45 +0530] [78126] [INFO] Starting gunicorn 20.1.0
[2023-06-28 16:29:45 +0530] [78126] [INFO] Listening at: http://127.0.0.1:5000 (78126)
[2023-06-28 16:29:45 +0530] [78126] [INFO] Using worker: sync
[2023-06-28 16:29:45 +0530] [78127] [INFO] Booting worker with pid: 78127
[2023-06-28 16:29:45 +0530] [78128] [INFO] Booting worker with pid: 78128
[2023-06-28 16:29:45 +0530] [78129] [INFO] Booting worker with pid: 78129
[2023-06-28 16:29:45 +0530] [78130] [INFO] Booting worker with pid: 78130
^C
[2023-06-28 16:30:25 +0530] [78126] [INFO] Handling signal: int
[2023-06-28 16:30:25 +0530] [78130] [INFO] Worker exiting (pid: 78130)
[2023-06-28 16:30:25 +0530] [78129] [INFO] Worker exiting (pid: 78129)
[2023-06-28 16:30:25 +0530] [78128] [INFO] Worker exiting (pid: 78128)
[2023-06-28 16:30:25 +0530] [78127] [INFO] Worker exiting (pid: 78127)
